# 라이브러리

In [1]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSemanticSegmentation, TrainingArguments, Trainer

import torch

import pandas as pd
from konlpy.tag import Mecab
import re
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1차 모델

In [12]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fear_data = survey[['content_id', 'fear']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fear_data = fear_data[fear_data['content_id'] != 0].reset_index().drop(columns=['index'])
fear_data['content_id'] = fear_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fear_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fear_data = pd.merge(fear_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fear_data['fear'] = fear_data['fear'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fear_data['content'] = fear_data['content'].apply(cleaned_content)

final_df = fear_data[['content', 'fear']]

# 라벨별 개수 확인
print(final_df['fear'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fear'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fear'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
print('\n')
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54129/815695750.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


fear
 0    4664
 1     292
-1      67
Name: count, dtype: int64


train_data fear
 0    3727
 1     234
-1      57
Name: count, dtype: int64

 test_data fear
 0    951
 1     47
-1      7
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4018
중복 제거 전 테스트 데이터셋: 1005




중복 제거 후 학습 데이터셋: 4003
중복 제거 후 테스트 데이터셋: 1001


In [13]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가 
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fear"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fear'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fear_model/check_point/try_1',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 10,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train() 

In [ ]:
trainer.save_model("./fear_model/try_1/fear_model1")
tokenizer.save_pretrained("./fear_model/try_1/fear_tokenizer1")

In [ ]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fear_model/try_1/fear_model1"
tokenizer_path = "./fear_model/try_1/fear_tokenizer1"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

  0%|          | 0/1001 [00:00<?, ?it/s]

100%|██████████| 1001/1001 [00:11<00:00, 86.68it/s]


,content,fear,pred
0,#1415_20221128_3인. 흐음.잘 열어보지 않으면,0,0
1,난.바보야.,0,0
2,들어가자마자 커피향이 씨게 낫다 좋앗다 하지만 인테리어는 그냥그랫다. 풀정도 되는듯.,0,0
3,3인. 스토리가 잘 감이 안 잡힌다!,0,0
4,1.5/3인. 활동성 있음.추락조심,0,0
...,...,...,...
999,나에겐 너무 어려웠던. 방탈짬바 있는 분들에게 추천,0,0
1000,2021. 12. 26. (3인) 각자 1인분씩 하고 뿌듯했던 테마.,0,0
1001,첫방 디버프로 초반에 절어버림. 볼것도 못보고 힌트썼는데 잘꾸며놓았다 . 살짝 장치...,0,0
1002,#5 - 2인 - 볼륨에 압도됨 - 히터도 방마다 빵빵함 - 다른 테마도 궁금해짐,0,0


In [ ]:
print(train_data['fear'].value_counts())
print(test_data['fear'].value_counts())

fear
 0    3714
 1     232
-1      57
Name: count, dtype: int64
fear
 0    947
 1     47
-1      7
Name: count, dtype: int64


In [ ]:
print('0포함', '\n', classification_report(test_data['fear'], test_data['pred']))
test_data2 = test_data[test_data['fear'] != 0]
print('0 제외', '\n', classification_report(test_data2['fear'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         7
           0       0.99      0.99      0.99       947
           1       0.71      0.85      0.78        47

    accuracy                           0.98      1001
   macro avg       0.57      0.61      0.59      1001
weighted avg       0.97      0.98      0.97      1001

0 제외 
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         7
           0       0.00      0.00      0.00         0
           1       0.89      0.85      0.87        47

    accuracy                           0.74        54
   macro avg       0.30      0.28      0.29        54
weighted avg       0.77      0.74      0.76        54



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [ ]:
test = ['공포도 겁나 큼', '개무서움', '무섭긴 한데 괜찮음', 
'스토리 구려', '겁나 무섭다고 그러더만 하나도 안 무서움', '무서운 건 없었어요.', '공포도 하나도 없음']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 공포도 겁나 큼, 평가: 2
---------- 

문장: 개무서움, 평가: 1
---------- 

문장: 무섭긴 한데 괜찮음, 평가: 2
---------- 

문장: 스토리 구려, 평가: 1
---------- 

문장: 겁나 무섭다고 그러더만 하나도 안 무서움, 평가: 2
---------- 

문장: 무서운 건 없었어요., 평가: 2
---------- 

문장: 공포도 하나도 없음, 평가: 2
---------- 



## 1차 결과
- 긍정 평가는 그나마 잘 해주는 것 같은데, 부정 평가가 여전히 박살남
- 데이터 증강! 데이터 증강!

# 2차 모델

In [ ]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fear_data = survey[['content_id', 'fear']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fear_data = fear_data[fear_data['content_id'] != 0].reset_index().drop(columns=['index'])
fear_data['content_id'] = fear_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fear_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fear_data = pd.merge(fear_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fear_data['fear'] = fear_data['fear'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fear_data['content'] = fear_data['content'].apply(cleaned_content)

final_df = fear_data[['content', 'fear']]

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54129/3995584655.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


In [ ]:
new_data = [('너무 무서움 진짜 레알', 1), ('무서워 무서워 무서워', 1), ('이게 바로 공포지', 1), ('이런 무서움은 두 번 다시 겪고 싶지 않아', 1),
('공포 1도 없는데?', -1), ('이런 걸 왜 무섭다고 함', -1), ('속았어요 속았어요 공포란 게 없어여', -1), ('공포도 없는 편', -1),
('쫄들도 할 수 있따', -1), ('스릴러라고 했지만 전혀 아닌', -1), ('공포도... 없는데...?', -1),
('공테 아님 공테 아님', -1), ('스릴러쟈나ㅏㅏ', 1), ('개무서움', 1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'fear'])
final_df = pd.concat([final_df, new_df])
final_df

,content,fear
0,오류장치 . . . 옵으로 다시 참여. 그 땐 문제도 못 풀었지만 100방 이상 쌓...,0
1,390. 조도 무슨일이야.,0
2,4인. 309번째 심연으로 떠납니다,0
3,4인. 극악의 조도. 그게 다.,0
4,"조도가 낮은게 아니라 없는 수준이라고 하던데, 확실히 느꼈다. 조도도 낮은데 관찰력...",0
...,...,...
9,스릴러라고 했지만 전혀 아닌,-1
10,공포도... 없는데...?,-1
11,공테 아님 공테 아님,-1
12,스릴러쟈나ㅏㅏ,1


In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(kiwi_clean)

In [ ]:
# 라벨별 개수 확인
print(final_df['fear'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fear'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fear'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

fear
 0    4664
 1     298
-1      75
Name: count, dtype: int64


train_data fear
 0    3727
 1     239
-1      64
Name: count, dtype: int64

 test_data fear
 0    939
 1     47
-1      7
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4030
중복 제거 전 테스트 데이터셋: 993


중복 제거 후 학습 데이터셋: 3932
중복 제거 후 테스트 데이터셋: 977


In [ ]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(35080, 128)

In [ ]:
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fear"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fear'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fear_model/check_point/try_2',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 30,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train 진행
trainer.train() 

  0%|          | 0/7380 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_52291/2167675541.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  7%|▋         | 500/7380 [01:28<19:37,  5.84it/s]

{'loss': 0.2504, 'learning_rate': 4.661246612466125e-05, 'epoch': 2.03}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_52291/2167675541.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 14%|█▎        | 1000/7380 [02:55<18:38,  5.71it/s]

{'loss': 0.1561, 'learning_rate': 4.3224932249322496e-05, 'epoch': 4.07}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_52291/2167675541.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 20%|██        | 1500/7380 [04:22<16:54,  5.79it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_52291/2167675541.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 95%|█████████▍| 7000/7380 [20:19<01:05,  5.81it/s]

{'loss': 0.0398, 'learning_rate': 2.574525745257453e-06, 'epoch': 28.46}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_52291/2167675541.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 7380/7380 [21:26<00:00,  5.74it/s]

{'train_runtime': 1286.0254, 'train_samples_per_second': 91.724, 'train_steps_per_second': 5.739, 'train_loss': 0.08109486044906988, 'epoch': 30.0}


TrainOutput(global_step=7380, training_loss=0.08109486044906988, metrics={'train_runtime': 1286.0254, 'train_samples_per_second': 91.724, 'train_steps_per_second': 5.739, 'train_loss': 0.08109486044906988, 'epoch': 30.0})

In [ ]:
trainer.save_model("./fear_model/try_2/fear_model2")
tokenizer.save_pretrained("./fear_model/try_2/fear_tokenizer2")

('./fear_model/try_2/fear_tokenizer2/tokenizer_config.json',
 './fear_model/try_2/fear_tokenizer2/special_tokens_map.json',
 './fear_model/try_2/fear_tokenizer2/vocab.txt',
 './fear_model/try_2/fear_tokenizer2/added_tokens.json')

In [ ]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fear_model/try_2/fear_model2"
tokenizer_path = "./fear_model/try_2/fear_tokenizer2"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    

100%|██████████| 977/977 [00:10<00:00, 97.20it/s] 


,content,fear,pred
0,장치 위주 협소 괜찮다 인테리어 문제 생각 쉽다,0,0
1,직원 친절 공포 없이 인테리어 잘 구현 소소 감성 테마 다양 종류 기출 문제 유형 ...,-1,-1
2,친구 친구,0,0
3,문제 장르 다양 문제 스토리 개연 좋다,0,0
4,우리 절 후폭풍 어마어마 감성 괜찮다 스토리 즐기다 시간 없다,0,0
...,...,...,...
987,나 너무 어렵다 방탈짬바 있다 추천,0,0
988,인 각자 하다 뿌듯하다 테마,0,0
989,첫 디버프로 초반 절다 보다 못 보다 힌트 쓰다 잘 꾸미다 살짝 장치 오류 아쉽다,0,0
990,인 볼륨 압도 히터 방 빵빵하다 다른 테마 궁금,0,0


In [ ]:
print('0포함', '\n', classification_report(test_data['fear'], test_data['pred']))
test_data2 = test_data[test_data['fear'] != 0]
print('0 제외', '\n', classification_report(test_data2['fear'], test_data2['pred']))

0포함 
               precision    recall  f1-score   support

          -1       0.75      0.86      0.80         7
           0       1.00      0.99      0.99       923
           1       0.86      0.91      0.89        47

    accuracy                           0.99       977
   macro avg       0.87      0.92      0.89       977
weighted avg       0.99      0.99      0.99       977

0 제외 
               precision    recall  f1-score   support

          -1       1.00      0.86      0.92         7
           0       0.00      0.00      0.00         0
           1       0.98      0.91      0.95        47

    accuracy                           0.91        54
   macro avg       0.66      0.59      0.62        54
weighted avg       0.98      0.91      0.94        54



/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [ ]:
test = ['공포도 겁나 큼', '개무서움', '무섭긴 한데 괜찮음', 
'스토리 구려', '겁나 무섭다고 그러더만 하나도 안 무서움', '무서운 건 없었어요.', '공포도 하나도 없음']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent}, 평가: {pred}')
    print("-"*10, '\n')

문장: 공포도 겁나 큼, 평가: 2
---------- 

문장: 개무서움, 평가: 1
---------- 

문장: 무섭긴 한데 괜찮음, 평가: 2
---------- 

문장: 스토리 구려, 평가: 1
---------- 

문장: 겁나 무섭다고 그러더만 하나도 안 무서움, 평가: 2
---------- 

문장: 무서운 건 없었어요., 평가: 2
---------- 

문장: 공포도 하나도 없음, 평가: 1
---------- 



# 3차 모델

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fear_data = survey[['content_id', 'fear']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fear_data = fear_data[fear_data['content_id'] != 0].reset_index().drop(columns=['index'])
fear_data['content_id'] = fear_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fear_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fear_data = pd.merge(fear_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fear_data['fear'] = fear_data['fear'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fear_data['content'] = fear_data['content'].apply(cleaned_content)

final_df = fear_data[['content', 'fear']]



/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_56779/1982797681.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


In [3]:
# 데이터 추가
new_data = [
    ('너무 무서움 진짜 레알', 1), ('무서워 무서워 무서워', 1), ('이게 바로 공포지', 1), ('이런 무서움은 두 번 다시 겪고 싶지 않아', 1),
    ('스릴러쟈나ㅏㅏ', 1), ('개무서움', 1), ('울면서 나옴', 1), ('너무 무섭잖아요', 1),
    ('정말 끔찍해서 눈 감고 싶다', 1), ('절대로 혼자 볼 수 없을 것 같아', 1), ('밤에 생각나서 잠이 안 온다', 1),
    ('저주받은 듯한 느낌이 들어 무섭다', 1), ('어둠 속에서 눈앞에 무서운 것이 보인다', 1), ('소름 끼치는 공간에 갇혀서 견딜 수 없다', 1),
    ('매일 밤 꿈에서도 공포에 시달리고 있다', 1), ('살면서 이렇게 무서운 순간은 처음이다', 1), ('공포에 사로잡혀 뒤처지는 기분이야', 1),
    ('이게 공포의 극치인 거 같아', 1),
    ('공포 1도 없는데?', -1), ('이런 걸 왜 무섭다고 함', -1), ('속았어요 속았어요 공포란 게 없어여', -1), ('공포도 없는 편', -1),
    ('쫄들도 할 수 있따', -1), ('스릴러라고 했지만 전혀 아닌', -1), ('공포도... 없는데...?', -1), ('공테 아님 공테 아님', -1), 
    ('극쪽임! 웃으면서 하고 나옴!', -1), ('극쫄이지만 전혀 무서운 건 없었어요', -1), ('공포? 어디?', -1), ('안무서움', -1),
    ('무섭지 않음', -1), ('전혀 안 무서움', -1), ('이건 누구나 할 수 있을 정도의 공포도였어요', -1), ('공포 없음', -1),
    ('공테 아님', -1), ('최강 극쫄도 쌉가능', -1), ('1도 안 무서움', -1),
    ('진짜 뭐가 무섭다는 건지 모르겠어', -1), ('이런 걸로 무서워하는 사람이 있을까?', -1), ('공포 소재라고 해서 완전 무서워질 줄 알았는데 그냥 그래', -1),
    ('이런 걸로 무서워하는 건 좀 어이없지 않아?', -1), ('공포는 뭔가 싶을 정도로 없어', -1), ('쫄아서 무서움을 모르는 건 아닌데 이건 아닌 듯', -1), 
    ('이런 걸로 놀라다니 참 웃긴 일이야', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'fear'])
final_df = pd.concat([final_df, new_df])
final_df
from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(kiwi_clean)
# 라벨별 개수 확인
print(final_df['fear'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fear'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fear'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

fear
 0    4664
 1     310
-1      93
Name: count, dtype: int64


train_data fear
 0    3723
 1     254
-1      77
Name: count, dtype: int64

 test_data fear
 0    916
 1     47
-1      6
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4054
중복 제거 전 테스트 데이터셋: 969


중복 제거 후 학습 데이터셋: 3953
중복 제거 후 테스트 데이터셋: 953


In [6]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fear"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fear'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fear_model/check_point/try_3',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 35,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# train 진행
trainer.train() 

trainer.save_model("./fear_model/try_3/fear_model3")
tokenizer.save_pretrained("./fear_model/try_3/fear_tokenizer3")




Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/8680 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54540/599019963.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  6%|▌         | 500/8680 [01:26<22:55,  5.95it/s]

{'loss': 0.2406, 'learning_rate': 4.711981566820277e-05, 'epoch': 2.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54540/599019963.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 12%|█▏        | 1000/8680 [02:52<21:57,  5.83it/s]

{'loss': 0.1151, 'learning_rate': 4.423963133640553e-05, 'epoch': 4.03}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54540/599019963.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 17%|█▋        | 1500/8680 [04:18<20:30,  5.84it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54540/599019963.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 98%|█████████▊| 8500/8680 [24:23<00:31,  5.69it/s]

{'loss': 0.0332, 'learning_rate': 1.0368663594470047e-06, 'epoch': 34.27}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_54540/599019963.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 8680/8680 [24:55<00:00,  5.80it/s]


{'train_runtime': 1495.2907, 'train_samples_per_second': 92.527, 'train_steps_per_second': 5.805, 'train_loss': 0.0649953881167047, 'epoch': 35.0}


('./fear_model/try_3/fear_tokenizer3/tokenizer_config.json',
 './fear_model/try_3/fear_tokenizer3/special_tokens_map.json',
 './fear_model/try_3/fear_tokenizer3/vocab.txt',
 './fear_model/try_3/fear_tokenizer3/added_tokens.json')

In [4]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fear_model/try_3/fear_model3"
tokenizer_path = "./fear_model/try_3/fear_tokenizer3"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

In [8]:
pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    
print('0포함', '\n', classification_report(test_data['fear'], test_data['pred']))
test_data2 = test_data[test_data['fear'] != 0]
print('0 제외', '\n', classification_report(test_data2['fear'], test_data2['pred']))

test = [
    '공포도 겁나 큼', '개무서움', '무섭긴 한데 괜찮음', '스토리 구려', 
    '겁나 무섭다고 그러더만 하나도 안 무서움', '무서운 건 없었어요.', '공포도 하나도 없음', 
    '뭐가 무서움?', '하나도 안 무서움', '공테 아닌 듯']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30, '\n')

100%|██████████| 953/953 [00:11<00:00, 85.97it/s]
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to co

0포함 
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         6
           0       0.99      0.99      0.99       900
           1       0.77      0.87      0.82        47

    accuracy                           0.98       953
   macro avg       0.59      0.62      0.60       953
weighted avg       0.98      0.98      0.98       953

0 제외 
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00         6
           0       0.00      0.00      0.00         0
           1       0.87      0.87      0.87        47

    accuracy                           0.77        53
   macro avg       0.29      0.29      0.29        53
weighted avg       0.77      0.77      0.77        53

문장: 공포도 겁나 큼 → 평가: 2
------------------------------ 

문장: 개무서움 → 평가: 2
------------------------------ 

문장: 무섭긴 한데 괜찮음 → 평가: 2
------------------------------ 

문장: 스토리 구려 → 평가: 1
------------------------------ 

문장: 겁나 무섭다고 그

# 4차

In [2]:
# labeling 된 데이터 불러오기
survey = pd.read_csv("./_data/survey.csv", index_col=0)
survey.fillna(0, inplace=True)
survey = survey.reset_index()

# 데이터 중에서 작업을 하고자 하는 라벨만 가져오기
fear_data = survey[['content_id', 'fear']]

# content_id 중에서 댓글이 같이 추가된 데이터 정리
fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)

# content_id 가 0인 데이터 제외 -> content_id 모두 int type 으로 변경 (추후 Merge 를 위함)
fear_data = fear_data[fear_data['content_id'] != 0].reset_index().drop(columns=['index'])
fear_data['content_id'] = fear_data['content_id'].astype(int)

# 전체 review data 불러오기
review_all = pd.read_csv("./_data/reviews.csv", index_col=0)

# 전체 review data 중에서 survey data에 있는 댓글만 가져오기
survey_content = review_all.loc[review_all['id'].isin(fear_data['content_id'])][['id', 'content']]
# merge 를 위해서 id 컬럼명 통일하기
survey_content.columns=['content_id', 'content']

# 통일된 content_id 를 기반으로 데이터 merge
fear_data = pd.merge(fear_data, survey_content)

# 추후 원활한 계산을 위해서 숫자 부분은 모두 int 로 바꿔줌
fear_data['fear'] = fear_data['fear'].astype(int)

# 최소한의 전처리
def cleaned_content(text):
    d = re.sub('\n', '. ', text) # 줄바꿈 > .
    d = re.sub('[^가-힣0-9a-zA-Z ]{2,}', ".", d) # 특수문자 두개 이상인거 .으로 변경
    return d

fear_data['content'] = fear_data['content'].apply(cleaned_content)

final_df = fear_data[['content', 'fear']]

# 데이터 추가
new_data = [
    ('너무 무서움 진짜 레알', 1), ('무서워 무서워 무서워', 1), ('이게 바로 공포지', 1), ('이런 무서움은 두 번 다시 겪고 싶지 않아', 1),
    ('스릴러쟈나ㅏㅏ', 1), ('개무서움', 1), ('울면서 나옴', 1), ('너무 무섭잖아요', 1),
    ('정말 끔찍해서 눈 감고 싶다', 1), ('절대로 혼자 볼 수 없을 것 같아', 1), ('밤에 생각나서 잠이 안 온다', 1),
    ('저주받은 듯한 느낌이 들어 무섭다', 1), ('어둠 속에서 눈앞에 무서운 것이 보인다', 1), ('소름 끼치는 공간에 갇혀서 견딜 수 없다', 1),
    ('매일 밤 꿈에서도 공포에 시달리고 있다', 1), ('살면서 이렇게 무서운 순간은 처음이다', 1), ('공포에 사로잡혀 뒤처지는 기분이야', 1),
    ('이게 공포의 극치인 거 같아', 1),
    ('공포 1도 없는데?', -1), ('이런 걸 왜 무섭다고 함', -1), ('속았어요 속았어요 공포란 게 없어여', -1), ('공포도 없는 편', -1),
    ('쫄들도 할 수 있따', -1), ('스릴러라고 했지만 전혀 아닌', -1), ('공포도... 없는데...?', -1), ('공테 아님 공테 아님', -1), 
    ('극쪽임! 웃으면서 하고 나옴!', -1), ('극쫄이지만 전혀 무서운 건 없었어요', -1), ('공포? 어디?', -1), ('안무서움', -1),
    ('무섭지 않음', -1), ('전혀 안 무서움', -1), ('이건 누구나 할 수 있을 정도의 공포도였어요', -1), ('공포 없음', -1),
    ('공테 아님', -1), ('최강 극쫄도 쌉가능', -1), ('1도 안 무서움', -1),
    ('진짜 뭐가 무섭다는 건지 모르겠어', -1), ('이런 걸로 무서워하는 사람이 있을까?', -1), ('공포 소재라고 해서 완전 무서워질 줄 알았는데 그냥 그래', -1),
    ('이런 걸로 무서워하는 건 좀 어이없지 않아?', -1), ('공포는 뭔가 싶을 정도로 없어', -1), ('쫄아서 무서움을 모르는 건 아닌데 이건 아닌 듯', -1), 
    ('이런 걸로 놀라다니 참 웃긴 일이야', -1)
]

new_df = pd.DataFrame(new_data, columns=['content', 'fear'])
final_df = pd.concat([final_df, new_df])

from kiwipiepy import Kiwi
kiwi = Kiwi()

def kiwi_clean(text):
    get_kiwi_pos = ['NNG', 'NP', 'NNP', 'MM', 'VV', 'VV-I', 'VV-R', 'VA', 'VA-I', 'VA-R', 'VCP', 'VCN', 'MAG', 'MAJ', 'XR']
    kiwi_lem = []
    for word in kiwi.tokenize(text):
        if word.tag in get_kiwi_pos:
            kiwi_lem.append(word.lemma)
    return ' '.join(kiwi_lem)

final_df['content'] = final_df['content'].apply(kiwi_clean)
# 라벨별 개수 확인
print(final_df['fear'].value_counts())
print('\n')
train_data = final_df.sample(frac=0.8, random_state=42).reset_index().drop(columns='index')
print('train_data', train_data['fear'].value_counts())
test_data = final_df.drop(train_data.index).reset_index().drop(columns='index')
print('\n', 'test_data', test_data['fear'].value_counts())
print('\n')
# 중복 데이터 제거(데이터 분리 후 중복이 생길 수 있어서 데이터 분리 후 중복 데이터 처리 진행)

# 데이터셋 개수 확인
print('중복 제거 전 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋: {}'.format(len(test_data)))
print('\n')
# 중복 데이터 제거
train_data.drop_duplicates(subset=['content'], inplace=True)
test_data.drop_duplicates(subset=['content'], inplace=True)
# 데이터셋 개수 확인
print('중복 제거 후 학습 데이터셋: {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋: {}'.format(len(test_data)))

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_59658/3684206452.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fear_data['content_id'] = fear_data['content_id'].apply(lambda x: x.split(',')[0] if len(str(x)) > 6 else x)


fear
 0    4664
 1     310
-1      93
Name: count, dtype: int64


train_data fear
 0    3723
 1     254
-1      77
Name: count, dtype: int64

 test_data fear
 0    916
 1     47
-1      6
Name: count, dtype: int64


중복 제거 전 학습 데이터셋: 4054
중복 제거 전 테스트 데이터셋: 969


중복 제거 후 학습 데이터셋: 3953
중복 제거 후 테스트 데이터셋: 953


In [3]:
# 토큰에 추가할 단어 -> '방탈출'이라는 도메인 지시기에 근거한 용어, 분리되어서는 안 되기 때문에 별도로 추가 작업 진행
addword = ['공테', '약공테', '감테', '창공', '갑툭튀', '삐딱', '삑', '꽝', '삑딱쾅', '삑딱쾅', '삑딱', '쫄', '극쫄',
            '극극쫄', '쫄팟', '쫄탱', '쫄보', '극', '약탱', '탱쫄', '극극극', '뉴비', '하드캐리', '극혐', '피지컬',
            '어거지', '뚝배기', '뚝문', '셀뚝', '억까', '트롤링', '트롤짓', '흙길', '풀길', '꽃길', '풀꽃', '꽃다발',
            '꽃밭', '웰메이드', '인생테마', '머글', '방린이', '방유아', '방세포', '방태아', '방탈러', '과몰입러', '옵저버',
            '리트', '연방', '혼방', '혼불', '워킹', '워크인', '장치방', '문제방', '직렬', '병렬', '육각형', '볼드', '볼드충', '에바',
            '가이드', '조도', '조명', '밝기', '어두움', '인테리어', '비주얼', '소품', '디자인',
            '스토리','기승전결','흐름도','결말','서사','이야기','유니버스','전개','시나리오', '개연성', '명료',
            '창의성','창의','신선','독특','참신','발상', '연출','짜임','사실감','구현','현실감','현장감', '활동성','활동력','활동량','움직임','반경',
            '규모','스케일','볼륨','사이즈','크기','넓이','공간감','분량', '공포','공테','무서움','담력','스릴러',
            '문제', '장치', '기계', '센서', '기구', '불친절',
            '메르헨', '커튼콜', '카르텔', '소우주', '풀문', '도고', '플래시', '나우히어', '나비효과', '몽중', '가이드라인',
            '연출력', '짜임새', '공포도', '공포감', '공포심', '약공테', '문제퀄']

# 사전학습된 bert 모델 사용
# num_labels 클래스에 대해서 훈련을 하기 위해서 num_labels=3 할당함, problem_type="multi_label_classification" 를 통해서 모델이 다중 레이블 분류에 해당함을 명시
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=3, problem_type="multi_label_classification")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

# token에 새로운 단어 추가
tokenizer.add_tokens(addword)
# token에 단어 추가후 기존 모델의 임베딩 레이어에 추가한 단어에 대한 임베딩 벡터가 없을 수 있기 때문
# 아래 코드를 통해서 토큰의 개수가 변했음을 모델에 알리고 모델의 임베딩 레이어를 조정하여 새로운 토큰을 수용할 수 있게 함
model.resize_token_embeddings(len(tokenizer))
# train content 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# test content 토큰화
tokenized_test_sentences = tokenizer(
    list(test_data['content']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 분류 모델에 넣기 위해서 1차원으로 구성된 세 개의 클래스를 2차원으로 재구성 후 label 로 투입
# -1(부정)=0, 0(중립)=1, 1(긍정)=2

train_label = []
for label in train_data["fear"].values:
    if label == -1:
        train_label.append([1., 0., 0.])
    elif label == 0:
        train_label.append([0., 1., 0.])
    elif label == 1:
        train_label.append([0., 0., 1.])

test_label = []
for label in test_data['fear'].values:
    if label == 0:
        test_label.append([0., 1., 0.])
    elif label == -1:
        test_label.append([1., 0., 0.])
    elif label == 1:
        test_label.append([0., 0., 1.])

# model 에 넣기 위한 dataset 생성 class
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
        
# bert 모델에 데이터가 들어갈 수 있게 만들어 둔 class를 활용하여 train, test dataset 생성
train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련시킬 때 사용되는 객체 설정 부분
training_args = TrainingArguments(
    output_dir = './fear_model/check_point/try_4',    # 모델과 훈련 중 생성되는 파일이 저장될 디렉토리 경로
    num_train_epochs = 50,              # 훈련 epoch 수
    per_device_train_batch_size = 16,    # 장치에 할당된 훈련 배치 크기
    per_device_eval_batch_size = 64,    # 장치에 할당된 평가 배치 크기, 모델을 평가할 때 사용되는 배치 크기
    logging_dir = './logs',             # 훈련 중 로그 파일이 저장될 디렉토리
    logging_steps = 500,                # 로그 출력 빈도, 500 step에 한 번씩 출력 예정
    save_total_limit = 2,               # 체크포인트 파일 저장 제한 수
)

# hugging face 의 trasformers 라이브러리를 사용하여 모델 훈련하고 관리하는 객체 설정 부분
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
# train 진행
trainer.train() 

trainer.save_model("./fear_model/try_4/fear_model4")
tokenizer.save_pretrained("./fear_model/try_4/fear_tokenizer4")




Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/12400 [00:00<?, ?it/s]/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_57185/968068253.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  4%|▍         | 500/12400 [01:27<34:16,  5.79it/s]

{'loss': 0.2394, 'learning_rate': 4.7983870967741937e-05, 'epoch': 2.02}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_57185/968068253.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  8%|▊         | 1000/12400 [02:55<33:53,  5.61it/s]

{'loss': 0.1136, 'learning_rate': 4.596774193548387e-05, 'epoch': 4.03}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_57185/968068253.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 12%|█▏        | 1500/12400 [04:24<32:45,  5.54it/s]

...

/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_57185/968068253.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
 97%|█████████▋| 12000/12400 [35:06<01:09,  5.73it/s]

{'loss': 0.006, 'learning_rate': 1.6129032258064516e-06, 'epoch': 48.39}


/var/folders/t4/314qsssd1p718wry9j0bh9000000gn/T/ipykernel_57185/968068253.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 12400/12400 [36:16<00:00,  5.70it/s]


{'train_runtime': 2176.3233, 'train_samples_per_second': 90.818, 'train_steps_per_second': 5.698, 'train_loss': 0.04828325083178859, 'epoch': 50.0}


('./fear_model/try_4/fear_tokenizer4/tokenizer_config.json',
 './fear_model/try_4/fear_tokenizer4/special_tokens_map.json',
 './fear_model/try_4/fear_tokenizer4/vocab.txt',
 './fear_model/try_4/fear_tokenizer4/added_tokens.json')

In [3]:
# 저장된 모델과 토크나이저를 불러오기

model_path = "./fear_model/try_4/fear_model4"
tokenizer_path = "./fear_model/try_4/fear_tokenizer4"

model = ElectraForSequenceClassification.from_pretrained(model_path)
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# 문장을 입력했을 때, 예측 값을 도출해주는 함수

from torch.nn.functional import softmax

# 함수 정의
def classify_text(text):
    # 문장을 토큰화하고 모델에 입력으로 전달
    text = cleaned_content(text)
    text = kiwi_clean(text)
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)

    # 소프트맥스 함수를 사용하여 확률값 계산
    probabilities = softmax(outputs.logits, dim=1)

    # 가장 높은 확률값에 해당하는 클래스 선택
    predicted_class = torch.argmax(probabilities).item()

    return predicted_class, probabilities

def change_num(num):
    if num == 0:
        return (-1)
    elif num == 1:
        return 0
    elif num == 2:
        return 1

In [5]:
pred_list = []
for sent in tqdm(test_data['content']):
    pred, _ = classify_text(sent)
    pred_list.append(pred)

test_data['pred'] = pred_list
test_data['pred'] = test_data['pred'].apply(change_num)
test_data
    
print('0포함', '\n', classification_report(test_data['fear'], test_data['pred']))
test_data2 = test_data[test_data['fear'] != 0]
print('0 제외', '\n', classification_report(test_data2['fear'], test_data2['pred']))

test = [
    '공포도 겁나 큼', '개무서움', '무섭긴 한데 괜찮음', '스토리 구려', 
    '겁나 무섭다고 그러더만 하나도 안 무서움', '무서운 건 없었어요.', '공포도 하나도 없음', 
    '뭐가 무서움?', '하나도 안 무서움', '공테 아닌 듯']

for sent in test:
    pred, _ = classify_text(sent)
    print(f'문장: {sent} → 평가: {pred}')
    print("-"*30, '\n')

100%|██████████| 953/953 [00:10<00:00, 87.79it/s]
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/envs/bert_electra/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _

0포함 
               precision    recall  f1-score   support

          -1       0.75      1.00      0.86         6
           0       0.99      0.99      0.99       900
           1       0.91      0.87      0.89        47

    accuracy                           0.99       953
   macro avg       0.89      0.96      0.91       953
weighted avg       0.99      0.99      0.99       953

0 제외 
               precision    recall  f1-score   support

          -1       0.86      1.00      0.92         6
           0       0.00      0.00      0.00         0
           1       1.00      0.87      0.93        47

    accuracy                           0.89        53
   macro avg       0.62      0.62      0.62        53
weighted avg       0.98      0.89      0.93        53

문장: 공포도 겁나 큼 → 평가: 2
------------------------------ 

문장: 개무서움 → 평가: 2
------------------------------ 

문장: 무섭긴 한데 괜찮음 → 평가: 2
------------------------------ 

문장: 스토리 구려 → 평가: 1
------------------------------ 

문장: 겁나 무섭다고 그